## <center> École Polytechnique de Montréal <br> Département Génie Informatique et Génie Logiciel <br>  LOG6308 - Systèmes de recommandations <br> </center>

## <center> TP1 -- Approches collaboratives : utilisateur-utilisateur, item-item, et agglomérative </center>

Le travail doit être fait en **équipe de deux**. 

## Identification de l'équipe: 02_eq28

### Groupe de laboratoire: 02

### Equipe numéro : 28

### Membres:

François Tourigny (2079718)

Gabriel Côté (2082508)

<br>

**Nature de la contribution:** Décrivez brièvement ce qui a été fait par chaque membre de l’équipe. Tous les membres sont censés contribuer au développement. Bien que chaque membre puisse effectuer différentes tâches, vous devez vous efforcer d’obtenir une répartition égale du travail. Soyez précis sur la contribution de chacun.

### Enoncé du TP

Ce notebook se trouve sous l'énoncé général du TP au lien [cours.polymtl.ca/MDesmarais/log6308/20251/Tp/tp1.html](https://cours.polymtl.ca/MDesmarais/log6308/20251/Tp/tp1.html).  Les données sont accessibles de ce lien.<br>

### Jeux de données

Vous avez 3 fichiers à votre disposition:

- 'Data/votes.csv': Matrice de données de 100 000 votes faits par 943 utilisateurs et portant sur 1682 items.
    - **user.id**: Indentifiant de l'utilisateur
    - **item.id**: Identifiant de l'item/film
    - **rating**: vote attribué à l'item par l'utilisateur
    - **timestamp**: Date d'enregistrement du vote (à ignorer pour ce TP) 
- 'Data/items.csv': Matrice de données sur les films
    - **movie.id**: Identifiant du film
    - **movie.title**: Nom du film
    - **release.date**: Date de sortie
    - **video.release.date**: Date de sortie de la video
    - **IMDb.URL**: Lien vers le film
    - les 19 autres champs sont les categories des films qui sont les suivantes:
        "unknown", "Action", "Adventure", "Animation", "Children's", "Comedy", "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror", "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"
- 'Data/u.csv': Matrice de données sur les utilisateurs
    - **id**: Identififiant de l'utilisateurs
    - **age**: Age de l'utilisateur
    - **gender**: Sexe de l'utilisateur
    - **job**: Emploi de l'utilisateur
    - **zip**: ZIP Code

Attention aux espaces et à la casse des differents champs. 

### Librairies permises

- numpy
- pandas
- seaborn
- matplotlib
- scipy


### Rédaction et remise du rapport

- Ce notebook constitue à la fois votre code et votre rapport. Il contient un squelette pour guider votre travail.

- Complétez directement le notebook, vous êtes libres de créer des nouvelles cellules de code ou de texte.

- <u>**IMPORTANT**</u> Remettez le ZIP contenant les données et le notebook sur Moodle avec le nom `MATRICULE1_MATRICULE2.ipynb` pour le notebook et `MATRICULE1_MATRICULE2.zip` pour le zip.


### CRITÈRES

- La démarche est valide et bien expliquée
- Les réponses sont correctes et commentées
- L'implémentation est performante et repose sur le calcul linéaire lorsqu'approprié
- La présentation est soignée et bien rédigée


### CODE D’HONNEUR

- __Règle 1__:  Le plagiat de code est bien évidemment interdit. Toute utilisation de code doit être référencée adéquatement. Vous __ne pouvez pas__ soumettre un code écrit par quelqu’un d’autre.

- __Règle 2__: Vous êtes libres de discuter des idées et des détails de mise en œuvre avec d'autres équipes. Cependant, vous ne pouvez en aucun cas consulter le code d'une autre équipe ou incorporer leur code dans votre TP.

- __Règle 3__:  Vous ne pouvez pas partager votre code publiquement (par exemple, dans un dépôt GitHub public) tant que le cours n'est pas fini.

## Question 1

### 1. Chargement des assets

#### 1.1. Chargement des librairies

In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import scipy as sp

#### 1.2. Chargement des données

Affichez tous les jeux de données

In [5]:
# TODO  --------------------------------- Préparation des Données ------------------------
## Chargement des votes
votes = pd.read_csv("data/ratings.csv")
# items 
items = pd.read_csv("data/items.csv")
# users
user = pd.read_csv("data/user.csv")
# End TODO

In [6]:
### TODO: Afficher les jeux de donnes
votes.head()
##End TODO

,user.id,item.id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [7]:
items.head()

,movie.id,movie.title,release.date,video.release.date,IMDb.URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [8]:
user.head()

,id,age,gender,job,zip
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


### 1.3. Calcul de performances

On désire calculer les performances prédictives de quelques approches qui donnent une base de référence: le vote aléatoire, le vote moyen, le vote moyen utilisateur, le vote moyen item et finalement, le vote attendu (moyenne du vote moyen utilisateur et item). Puis rapporter l'erreur quadratique moyenne et l'erreur absolue moyenne (1) sans validation croisée et (2) avec validation croisée de 5 replis. Et enfin, déterminer si la différence entre l'erreur quadratique moyenne obtenue par le vote moyen item et le vote moyen utilisateur est statistiquement significative par un test d'hypothèse et en affichant les résultats visuellement. Pour cela, on procède en plusieurs étapes.


#### 1.3.1. Prétrairements
L’objectif ici est de créer une matrice utilisateurs-items qui, à chaque utilisateur, associe les votes qu’il a attribués aux films.

In [9]:
## Conversion du Pandas Datafram en Matrice Utilisateur Item : MUI
MUI = votes.pivot(index="user.id", columns="item.id", values="rating")
MUI.head()

item.id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user.id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
## Convertir le DF à une matrice numpy
MUI_numpy = MUI.to_numpy()
MUI_numpy_flat = MUI_numpy.reshape(-1)


### 1.3.2. Fonctions de calcul d'erreurs

Afin de calculer l'erreur quadratique moyenne et l'erreur absolue moyenne vous devez utiliser les fonctions ci-dessous. Completez leurs codes.

In [11]:
def RMSE_mat(y_pred, y_true):
    return np.sqrt(np.nanmean((y_pred - y_true) ** 2))

In [12]:
def MAE_mat(y_pred, y_true):
    return np.nanmean(np.abs(y_pred - y_true))

#### 1.3.3. Calcul des performances sans validation croisée

La validation croisée est une technique qui permet d'évaluer les performances d'un modèle/système sur de nouvelles données. Elle permet d'évaluer la capacité d'un modèle à généraliser la comprehension d’un problème. C’est-à-dire, à faire de bonnes prédictions sur des données qu’il n’a pas encore vues. Dans cet exercice, elle nous permet de savoir si l'approche employée permet de prédire correctement les votes attendues. Mais avant d'y arriver, dans cet exercice, afin de bien comprendre l'intérêt de cette technique, il est intéressant de commencer par une approche sans validation croisée (on utilise toutes les données).

Le code ci-dessous permet de calculer les prédictions de votes sans validation croisée. Complétez-le et commentez les résultats.

In [13]:
# TODO
## Vote aleatoire
def vote_aleatoire():
    return np.random.randint(6)

## Vote moyen
def vote_moyen(df):
    return np.nanmean(df)

## Vote moyen utilisateur
def vote_moyen_utilisateurs(df):
    return np.nanmean(df, axis=1)

## Vote moyen item
def vote_moyen_items(df):
    return np.nanmean(df, axis=0)

## Vote Moyen Attendu (Moyenne du vote moyen utilisateur et item)
def vote_moyen_attendu(df):
    return (vote_moyen_utilisateurs(df) + vote_moyen_items(df))/2


#### 1.3.4. Calcul des performances avec validation croisée

Les cellules ci-dessous contiennent le code pour faire la prediction de votes et le calcul des erreurs quadratiques moyennes et erreurs absolue moyenne. Completez le et commentez les resultats.

In [14]:
## Création des indices pour les valeurs différentes de np.nan
indices = np.arange(0, MUI_numpy.shape[0]*MUI_numpy.shape[1])
indices_na = indices[~np.isnan(MUI_numpy_flat)]

## Split Train Test des indices
nbre_replis = 5
np.random.shuffle(indices_na)

idx_split = np.split(indices_na, nbre_replis)

print(indices_na.shape)

(100000,)


In [15]:
## Je construis ma liste d'indice train et test
# Pour faire une cross validation à 5 replis il suffit de remplacer 0 par i
# et itérer de 0 à 4
list_RMSE = []
list_MAE = []

for i in range(nbre_replis):

    MUI_numpy_train_idx = np.concatenate(idx_split[:i] + idx_split[i+1:]) # Liste d'indices train
    MUI_numpy_test_idx = idx_split[i] # Liste d'indices test

    ## J'enlève les valeurs de test de la matrice d'entrainement, et vice versa
    MUI_numpy_train = np.copy(MUI_numpy)
    MUI_numpy_test = np.copy(MUI_numpy)
    
    row_train, col_train = zip(*[(idx//MUI_numpy.shape[1], idx%MUI_numpy.shape[1]) for idx in MUI_numpy_train_idx])
    row_test, col_test = zip(*[(idx//MUI_numpy.shape[1], idx%MUI_numpy.shape[1]) for idx in MUI_numpy_test_idx])

    
    #  Je redonne la structure de matrice aux ensembles de test et d'entrainement
    MUI_numpy_test[row_train, col_train] = np.nan
    MUI_numpy_train[row_test, col_test] = np.nan


    # On s'assure d'avoir les bonnes dimensions
    #print(MUI_numpy_train[~np.isnan(MUI_numpy_train)].shape)
    #print(MUI_numpy_test[~np.isnan(MUI_numpy_test)].shape)

    # Documentation pour np.nanmean : https://numpy.org/doc/stable/reference/generated/numpy.nanmean.html
        
    # --------------------------------- Prédictions des Valeurs Test ---------------------------------
    # Prédiction des votes aléatoires entre 1 et 5 avec une distribution uniforme d'entiers
    votes_alea_pred      = np.full(MUI_numpy_test.shape, vote_aleatoire())

    # Vote Moyen
    # Prédiction des votes grâce à la moyenne des votes de la matrice d'entrainement
    votes_moyenne_pred   = np.full(MUI_numpy_test.shape, vote_moyen(MUI_numpy_train))

    # Vote Moyen Utilisateur
    # Prédiction des votes grâce à la moyenne des votes par utilisateur de la matrice d'entrainement
    votes_moyenne_U_pred = np.tile(vote_moyen_utilisateurs(MUI_numpy_train)[:, np.newaxis], (1, MUI_numpy_test.shape[1]))

    # Vote Moyen Utilisateur
    # Prédiction des votes grâce à la moyenne des votes par item de la matrice d'entrainement
    votes_moyenne_I_pred = np.tile(vote_moyen_items(MUI_numpy_train), (MUI_numpy_test.shape[0], 1))

    # Vote Moyen Attendu (Moyenne du vote moyen utilisateur et item)
    moyenne_U_repeat = np.tile(vote_moyen_utilisateurs(MUI_numpy_train)[:, np.newaxis], (1, MUI_numpy_test.shape[1]))
    moyenne_I_repeat = np.tile(vote_moyen_items(MUI_numpy_train), (MUI_numpy_test.shape[0], 1))

    votes_moyenne_A_pred = (moyenne_U_repeat + moyenne_I_repeat)/2

    # Calcul des RMSE
    list_matrices = [votes_alea_pred, votes_moyenne_pred, votes_moyenne_U_pred, votes_moyenne_I_pred, votes_moyenne_A_pred]

    list_RMSE.append(np.array([RMSE_mat(mat, MUI_numpy_test) for mat in list_matrices]))
    list_MAE.append(np.array([MAE_mat(mat, MUI_numpy_test) for mat in list_matrices]))

list_RMSE = np.array(list_RMSE)
list_MAE = np.array(list_MAE)

print("Résultats pour le vote aléatoire sur l'ensemble de test :")
print("RMSE: ", list_RMSE.mean(axis=0)[0])
print("MAE: ", list_MAE.mean(axis=0)[0],'\n')

print("Résultats pour le vote moyen sur l'ensemble de test :")
print("RMSE: ", list_RMSE.mean(axis=0)[1])
print("MAE: ", list_MAE.mean(axis=0)[1],'\n')

print("Résultats pour le vote moyen utilisateur sur l'ensemble de test :")
print("RMSE: ", list_RMSE.mean(axis=0)[2])
print("MAE: ", list_MAE.mean(axis=0)[2],'\n')

print("Résultats pour le vote moyen item sur l'ensemble de test :")
print("RMSE: ", list_RMSE.mean(axis=0)[3])
print("MAE: ", list_MAE.mean(axis=0)[3],'\n')

print("Résultats pour le vote moyen attendu sur l'ensemble de test :")
print("RMSE: ", list_RMSE.mean(axis=0)[4])
print("MAE: ", list_MAE.mean(axis=0)[4],'\n')

C:\Users\gabco\AppData\Local\Temp\ipykernel_56588\2892671948.py:16: RuntimeWarning: Mean of empty slice
  return np.nanmean(df, axis=0)


Résultats pour le vote aléatoire sur l'ensemble de test :
RMSE:  1.9756679824019518
MAE:  1.67288 

Résultats pour le vote moyen sur l'ensemble de test :
RMSE:  1.1256406251903333
MAE:  0.9446997542499996 

Résultats pour le vote moyen utilisateur sur l'ensemble de test :
RMSE:  1.0423591494556348
MAE:  0.8354752801547415 

Résultats pour le vote moyen item sur l'ensemble de test :
RMSE:  1.0224394680298379
MAE:  0.8155235416556588 

Résultats pour le vote moyen attendu sur l'ensemble de test :
RMSE:  0.981656487184849
MAE:  0.7941590907324825 



### 1.4. Analysez des resultats

On souhaite analyse et valider les resultats obtenus

#### 1.4.1. Analyse comparatives des resultats

Faites une analyse comparative des résultats  obtenus sans et avec la validation croisée. À quoi s'attendaient-t-on ? Qu'est-ce qu'on a obtenu? Pourquoi ?

In [16]:
#TODO Analayse

#End TODO

#### 1.4.2. Test d'hypothèse
On souhaites déterminer si la différence entre l'erreur quadratique moyenne obtenue par le vote moyen item et le vote moyen utilisateur est statistiquement significative. Faites un test d'hypothèse à cet fin.

In [17]:
# TODO
## t-Test:

# End TODO

## Question 2 - Filtres Collaboratifs

On désire calculer les erreurs des prédictions des approches de filtres collaboratifs item-item et utilisateur-utilisateur. Pour cela, on se propose d'utiliser les fonctions ci-dessous qui permettent de:
 - Calculer la similarité cosinus entre deux vecteurs/matrices ;
 - Predire les votes par l'approche filtrage collaboratifs .

Completez les!

In [18]:
## Similarité Cosinus entre les vecteurs ligne de la matrice A
#  et les vecteurs ligne de la matrice B
def cosinus_matrices(A, B):
    A = A.copy()
    B = B.copy()

    # Remplace les nan par 0, pour ne pas rajouter + de NAN après le produit matriciel
    A = np.nan_to_num(A, nan=0.0)
    B = np.nan_to_num(B, nan=0.0)

    # Matrice colonne des normes de chacuns des vecteurs de A
    col_norm_A = np.linalg.norm(A, axis=1, keepdims=True)

    # Matrice ligne des normes de chacuns des vecteurs de B
    col_norm_B = np.linalg.norm(B, axis=1, keepdims=True).T 

    dot_product = np.dot(A, B.T)
    norm_product = col_norm_A * col_norm_B
    cos_similarity = dot_product / norm_product
    cos_similarity = np.nan_to_num(cos_similarity, nan=0.0)

    return cos_similarity

In [19]:
def filtre_collaboratif(V, W):

    # Crée une matrice booléenne de la même taille que V. Les éléments qui ne sont pas NaN dans V 
    # (c'est-à-dire les éléments pour lesquels une note existe) sont marqués par 1, et les NaN par 0. 
    # Cette matrice sera utilisée pour s'assurer que seules les notes existantes contribuent aux prédictions.
    Bool_m = (~np.isnan(V))*1 

    V = V.copy()
    W = W.copy()

    # Les valeurs NaN dans V et W sont remplacées par 0. Cela permet d'effectuer des opérations matricielles numpy sans erreur.
    V = np.nan_to_num(V, nan=0.0)
    W = np.nan_to_num(W, nan=0.0)

    # Calcule la somme des similarités absolues pour chaque utilisateur, mais seulement pour les items qu'ils ont notés (grâce à Bool_m). 
    K = np.abs(W).dot(Bool_m.T)
    
    # Calcule les notes prédites. 
    numerateur = W.dot(V)
    K[K == 0] = 1
    V_pred = numerateur / K
    V_pred = V_pred * (1 - Bool_m) + V * Bool_m

    return V_pred

Maintenant, pour le calcul des differents erreur, procédez par étapes et rapportez, à chaque étape, l'erreur quadratique moyenne et l'erreur absolue moyenne à partir de la matrice de prédiction obtenue à l'aide de la matrice cosinus de similarité.

M_pred = MUI x M_cos

## 2 -a) Prédiction avec tous les utilisateurs/items 

Prediction (sans voisins rapprochés) et sans correction pour biais utilisateur/item (sans normaliser en soustrayant la moyenne utilisateur/item)

In [24]:
# TODO

## Approche Utilisateur
MUI_filled = np.nan_to_num(MUI_numpy, nan=0.0)
M_cosU = cosinus_matrices(MUI_filled, MUI_filled)

M_pred = M_cosU @ MUI_filled
row_sums = np.sum(M_cosU, axis=1, keepdims=True)
M_pred_normalized_U = M_pred / row_sums

mask = ~np.isnan(MUI_numpy)
M_pred_normalized_U[mask] = M_pred_normalized_U[mask]
M_pred_normalized_U

#End TODO

array([[2.42492083e+00, 7.58241451e-01, 4.44433098e-01, ...,
        5.37552717e-04, 6.51021154e-03, 7.36117851e-03],
       [1.88253580e+00, 3.25506086e-01, 3.02482424e-01, ...,
        3.36814306e-03, 2.77008816e-03, 1.91219324e-03],
       [1.32849217e+00, 2.90918284e-01, 2.36176026e-01, ...,
        8.35623444e-03, 2.61921448e-03, 1.18219930e-03],
       ...,
       [2.45321901e+00, 4.48412395e-01, 3.96938958e-01, ...,
        1.22401650e-03, 4.00566986e-03, 3.02918177e-03],
       [2.03994198e+00, 5.87603968e-01, 3.03290502e-01, ...,
        2.08310984e-03, 5.33460874e-03, 3.31510554e-03],
       [2.51325405e+00, 8.82071088e-01, 4.83121150e-01, ...,
        0.00000000e+00, 7.53072560e-03, 7.06815298e-03]])

In [25]:
rmse_U = RMSE_mat(M_pred_normalized_U, MUI_numpy)
mae_U = MAE_mat(M_pred_normalized_U, MUI_numpy)
print("RMSE: ", rmse_U)
print("MAE: ", mae_U,'\n')

np.float64(2.800621511592025)

In [17]:
# TODO

## Approche Item
MUI_filled = np.nan_to_num(MUI_numpy, nan=0.0).T

M_cosI = cosinus_matrices(MUI_filled, MUI_filled)
M_pred = M_cosI @ MUI_filled

row_sums = np.sum(M_cosI, axis=1, keepdims=True)
M_pred_normalized_I = M_pred / row_sums

mask = ~np.isnan(MUI_numpy.T)
M_pred_normalized_I[mask] = MUI_numpy.T[mask]

M_pred_normalized_I

#End TODO

array([[5.        , 4.        , 0.10855542, ..., 5.        , 0.39591598,
        0.76778773],
       [3.        , 0.15536591, 0.08034748, ..., 0.09172743, 0.32956602,
        5.        ],
       [4.        , 0.21683346, 0.10507397, ..., 0.12464073, 0.28190757,
        0.75741136],
       ...,
       [0.15524015, 0.36303179, 0.59657341, ..., 0.04157438, 0.2562126 ,
        0.        ],
       [1.11420477, 0.14495846, 0.09008272, ..., 0.10254288, 0.34691859,
        0.84284274],
       [1.5037109 , 0.11693814, 0.03229978, ..., 0.0868939 , 0.21783623,
        0.8583227 ]])

## 2 -b) Correction pour biais utilisateur/item. 

Vous pouvez prendre l'ensemble des les valeurs observées dans le calcul des moyennes items et utilisateurs (des précisions sur ce point seront données en classe — voir aussi le bonus)


> Ici le biais utilisé est le Vote moyen.  
Attention (Moyenne du vote moyen utilisateur et item)

In [40]:
#TODO

## Calcul de la moyenne attendue
votes_moyenne_U_pred = ?

# Vote Moyen Utilisateur
# Prédiction des votes grâce à la moyenne des votes par item de la matrice d'entrainement
votes_moyenne_I_pred = ?

# Vote Moyen Attendu (Moyenne du vote moyen utilisateur et item)
moyenne_U_repeat = ?
moyenne_I_repeat = ?

# Notre Baseline
R_moy = ?

#End TODO

NB : Il est possible d'utiliser la matrice de vote moyen utilisateur ou item au lieu d'utiliser le vote attendu. Néanmoins le vote attendu donne de meilleurs résultats.

In [ ]:
#TODO : Approche Utilisateur
## Calculer la matrice de vote Item-Item sans biais
MUI_WB_U_U = MUI - ?

#Code


#End TODO

In [ ]:
#TODO : Approche Item
## Calculer la matrice de vote Item-Item sans biais
MUI_WB_I_I = MUI.T - ?

#Code



#End TODO

## 2 -c) Ajout de voisins rapprochés

Avec k=100 voisins rapprochés

In [ ]:
## Approche Utilisateur


In [ ]:
## Approche Item
